# Path Finder

## Setup Spark environment

In [ ]:
%%local
import os
username = os.environ['JUPYTERHUB_USER']
get_ipython().run_cell_magic('configure', line="-f", cell='{ "name":"%s-final-path-finder", "executorMemory":"4G", "executorCores":4, "numExecutors":10, "driverMemory": "4G" }' % username)

In [ ]:
%%send_to_spark -i username -t str -n username

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

## Import data from HDFS

In [ ]:
routes = spark.read.csv('/data/sbb/csv/timetable/routes/2019/05/07/routes.csv', header=True )
stops = spark.read.orc('/data/sbb/orc/geostops')

In [ ]:
walking_edges = spark.read.parquet('/user/%s/final/parquet/walking_edges' % username)
transport_edges = spark.read.parquet('/user/%s/final/parquet/transport_edges' % username)

In [ ]:
transport_edges.columns

## Shortest Path

In [ ]:
inward_walking_edges = walking_edges.groupby('end_vertex').agg({'start_vertex': 'collect_set'})\
                                                              .withColumnRenamed('collect_set(start_vertex)', 'start_vertices')\
                                                              .toPandas().set_index('end_vertex').to_dict()['start_vertices']
walking_edge_duration = walking_edges.toPandas()
walking_edge_duration['key'] = walking_edge_duration.apply(lambda x: (x['start_vertex'], x['end_vertex']), axis=1)
walking_edge_duration = walking_edge_duration.set_index('key').to_dict()['duration']

In [ ]:
edges = transport_edges.filter('weekday == "monday"')

In [ ]:
# This function assumes a (spark) dataframe with columns 'start_vertex', 'end_vertex', 'start_time', 'duration', 'route_id' 
# edges is a dataframe that represents the edges of the graph, 
# edges should be sorted by order of descending starting time
# inward_walking_edges is a map route_id -> list(route_id), representing the nodes from which we can walk to a given node
# walking_edge_duration is a map (route_id, route_id) -> float, representing the duration of the walk
# end_time should be in the format 'HH:MM'
def latest_departure_paths(edges, end_stop, end_time, inward_walking_edges, walking_edge_duration):
    
    # This function computes whether edge (u, v, t, dur, route_id) can be taken
    # returns update to node u if possible, and None otherwise
    def edge_valid(u, v, time, dur, route_id):
        
        # get curren best and best way to get to v
        cur_best = next_transfer.get(u)
        next_edge = next_transfer.get(v)
        
        
        if next_edge: # needs to be defined
            time_v, next_v, route_id_v, dur_v, seq_v = next_edge
            
            # if provided edge is a walking edge
            if route_id == 'walking':

                if  route_id_v == 'walking' and dur_v + dur <= max_walking_time:
                    time = time_v - dur - transfer_time
                    
                    if (not cur_best) or time > cur_best[0]:
                        return (time, next_v, route_id, dur + dur_v, [u] + seq_v)
                    
                elif route_id_v != 'walking':
                    time = time_v - dur - transfer_time
                    
                    if (not cur_best) or time > cur_best[0]:
                        return (time, v, route_id, dur, [u, v])
                      
            
            elif (route_id == route_id_v or v == end_stop) and time + duration <= time_v: # same route or last_stop
                
                if (not cur_best) or time > cur_best[0]: # can we improve
                    return (time, next_v, route_id, dur + dur_v, [u] + seq_v)
            
            
            elif time + duration + transfer_time <= time_v: # transfer at node_v
                
                if (not cur_best) or time > cur_best[0]:
                    return (time, v, route_id, dur, [u, v]) 
        
        # In any other case no update can be made
        return None
                
    def time_to_minutes(timestamp):
        return int(timestamp[:2]) * 60 + int(timestamp[3:5])
    
    transfer_time = 2 # at least 2 minutes to transfer
    max_walking_time = 10

    next_transfer = {} # stores for each node u the node v 
                       # where the next transfer happens format (departure_time, v, route_id to v, duration)
    end_time = time_to_minutes(end_time)
    start_time = end_time - 120 # look only at edges departing at most 2 hours before end_time
    
    next_transfer[end_stop] = (end_time, end_stop, None, 0, [end_stop])
    
    edges = edges.filter(F.col('start_time').between(start_time, end_time)).toPandas().sort_values(by=['start_time'], ascending=False).to_numpy()

    for row in edges:
        
        start_vertex, end_vertex, start, duration, route_id = row[:5]
        
        update = edge_valid(start_vertex, end_vertex, start, duration, route_id)
        if update:
            next_transfer[start_vertex] = update
                    
            w_start_vertex = start_vertex
            for w_end_vertex in inward_walking_edges.get(w_start_vertex, []):
                duration = walking_edge_duration[(w_start_vertex, w_end_vertex)]
                w_start = start 
                update = edge_valid(w_start_vertex, w_end_vertex, w_start, duration, 'walking')
                if update:
                    next_transfer[w_start_vertex] = update

    return next_transfer

In [ ]:
import time
start = time.time()
next_transfer = latest_departure_paths(edges, '8591221', '20:00', inward_walking_edges, walking_edge_duration)
end = time.time() 
print(end - start)

In [ ]:
route_id_to_route_name = routes.withColumn('route_name', F.concat(F.col('route_desc'), F.lit(' '), F.col('route_short_name')))\
                               .select('route_id', 'route_desc', 'route_short_name', 'route_name').toPandas()
route_id_to_route_name_dict = route_id_to_route_name.set_index('route_id').to_dict()['route_name']
route_id_to_route_name_dict['walking'] = 'walk'

In [ ]:
stop_id_to_stop = stops.select('stop_id', 'stop_name').toPandas()
stop_id_to_stop_dict = stop_id_to_stop.set_index('stop_id').to_dict()['stop_name']

In [ ]:
def minutes_to_timestamp(minutes):
    return str(minutes // 60) + ':' + str(minutes % 60).zfill(2)
    
    
def reconstruct_journey(next_transfer, start_stop):
    time, next_stop, route_id, duration, stop_seq = next_transfer[start_stop]
    journey = []
    while next_stop != start_stop:
        journey.append(
            (minutes_to_timestamp(time),
             stop_id_to_stop_dict[start_stop], 
             route_id_to_route_name_dict[route_id],
             duration,
             'Ride %d stops' % (len(stop_seq)-1),
             minutes_to_timestamp(time + duration),
             stop_id_to_stop_dict[next_stop])
        )
        
        start_stop = next_stop
        time, next_stop, route_id, duration, stop_seq = next_transfer[next_stop]
    return journey

In [ ]:
def pretty_print_journey(journey):
    journey_str = '\n'.join(["%s : %s\n|\n| %s (%s min) %s\n|\n%s : %s\n" % segment for segment in journey])
    return journey_str

In [ ]:
journey = reconstruct_journey(next_transfer, '8591368')

In [ ]:
journey

In [ ]:
print(pretty_print_journey(journey)) # Correct

## Send to local

Send transfer dataframe to local

In [ ]:
%%spark -o next_transfer_df -n -1

In [ ]:
%%local
next_transfer_df.head()

Reconstruct a dict for transfers to keep old code

In [ ]:
%%local
next_transfer = {e[0]: tuple(e[1:]) for e in next_transfer_df.to_numpy()}

Send route id to route name mapping to local

In [ ]:
%%spark -o route_id_to_route_name -n -1
route_id_to_route_name = routes.withColumn('route_name', F.concat(F.col('route_desc'), F.lit(' '), F.col('route_short_name')))\
                               .select('route_id', 'route_desc', 'route_short_name', 'route_name')

In [ ]:
%%local
route_id_to_route_name_dict = route_id_to_route_name.set_index('route_id').to_dict()['route_name']
route_id_to_route_name_dict['walking'] = 'walk'

Send stop id to stop name mapping to local

In [ ]:
%%spark -o stop_id_to_stop -n -1
stop_id_to_stop = stops.select('stop_id', 'stop_name')

In [ ]:
%%local
stop_id_to_stop_dict = stop_id_to_stop.set_index('stop_id').to_dict()['stop_name']

In [ ]:
%%local
def pretty_print_journey(journey):
    journey_str = '\n'.join(["%s : %s\n|\n| %s (%s min)\n|\n%s : %s\n" % segment for segment in journey])
    return journey_str

In [ ]:
%%local
print(pretty_print_journey(reconstruct_journey(next_transfer, '8590883'))) # almost same as sbb

In [ ]:
%%local
print(pretty_print_journey(reconstruct_journey(next_transfer, '8591353'))) # almost same as sbb

In [ ]:
%%local
print(pretty_print_journey(reconstruct_journey(next_transfer, '8590788'))) # not in sbb

In [ ]:
%%local
print(pretty_print_journey(reconstruct_journey(next_transfer, '8591368'))) # Correct